# [Exploration 11] 뉴스 요약봇 만들기
## 프로젝트 설명 



## 프로젝트 목표


## 01. load data
- 뉴스기사 데이터(newssummarymore.csv)사용 : [링크](https://github.com/sunnysai12345/News_Summary)

In [2]:
import nltk
nltk.download('stopwords')

import numpy as np
import pandas as pd
import os
import re
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from bs4 import BeautifulSoup 
from tensorflow.keras.preprocessing.text import Tokenizer 
from tensorflow.keras.preprocessing.sequence import pad_sequences
import urllib.request

[nltk_data] Downloading package stopwords to /home/aiffel-
[nltk_data]     dj26/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/sunnysai12345/News_Summary/master/news_summary_more.csv", filename="news_summary_more.csv")
data = pd.read_csv('news_summary_more.csv', encoding='iso-8859-1')

In [4]:
data.head()

,headlines,text
0,upGrad learner switches to career in ML & Al w...,"Saurav Kant, an alumnus of upGrad and IIIT-B's..."
1,Delhi techie wins free food from Swiggy for on...,Kunal Shah's credit card bill payment platform...
2,New Zealand end Rohit Sharma-led India's 12-ma...,New Zealand defeated India by 8 wickets in the...
3,Aegon life iTerm insurance plan helps customer...,"With Aegon Life iTerm Insurance plan, customer..."
4,"Have known Hirani for yrs, what if MeToo claim...",Speaking about the sexual harassment allegatio...


In [5]:
data.sample(10)

,headlines,text
9142,Wiped-out US hedge fund manager confesses his ...,A US-based hedge fund manager has apologised f...
13386,Couldn't spell 'entrepreneur' when I heard it ...,"OYO's Founder and CEO, 24-year-old Ritesh Agar..."
72925,"Team declares at 18-1, loses match but wins title","Carew Cricket Club, a Welsh club, won their le..."
73935,Mark Wahlberg named world's highest paid actor...,Deepwater Horizon actor Mark Wahlberg has topp...
70121,Happy b'day to my favouritest human: Akshay on...,Akshay Kumar shared a picture of his son Aarav...
75513,Haven't received formal order: Pahlaj on Censo...,Ousted Censor Board Chief Pahlaj Nihalani has ...
97279,How did hair removal become a norm for women?,It is believed that Ancient Egyptians removed ...
68677,HC relief for Mumbai eatery which billed Ã¢ÂÂ...,The Bombay High Court has restrained governmen...
78356,"Rolling Stone features Trudeau, asks why can't...",US magazine Rolling Stone featured Canadian PM...
7280,Top 10 fastest-growing cities from 2019 to 203...,All the top 10 fastest growing cities in the w...


## 02. 데이터 전처리
- 중복샘플, NULL 처리

In [8]:
print('전체 data 길이 :', len(data))
print('unique한 headline 수:', data.headlines.nunique())
print('unique한 text 수:', data.text.nunique())

전체 data 길이 : 98401
unique한 headline 수: 98280
unique한 text 수: 98360


- text를 기준으로 중복데이터를 제거

In [10]:
data.drop_duplicates(subset=['text'], inplace = True)
print('전체 data 길이 :', len(data))
print('unique한 headline 수:', data.headlines.nunique())

전체 data 길이 : 98360
unique한 headline 수: 98262


- null 확인 : 없음!

In [11]:
print(data.isnull().sum())

headlines    0
text         0
dtype: int64


- text normalization(텍스트 정규화)
    - it'll == it will : 문법을 통일화해준다.

Step 1. 데이터 수집하기

Step 2. 데이터 전처리하기 (추상적 요약)
실습에서 사용된 전처리를 참고하여 각자 필요하다고 생각하는 전처리를 추가 사용하여 텍스트를 정규화 또는 정제해 보세요. 만약, 불용어 제거를 선택한다면 상대적으로 길이가 짧은 요약 데이터에 대해서도 불용어를 제거하는 것이 좋을지 고민해보세요.

Step 3. 어텐션 메커니즘 사용하기 (추상적 요약)
일반적인 seq2seq보다는 어텐션 메커니즘을 사용한 seq2seq를 사용하는 것이 더 나은 성능을 얻을 수 있어요. 실습 내용을 참고하여 어텐션 메커니즘을 사용한 seq2seq를 설계해 보세요.

Step 4. 실제 결과와 요약문 비교하기 (추상적 요약)
원래의 요약문(headlines 열)과 학습을 통해 얻은 추상적 요약의 결과를 비교해보세요.

Step 5. Summa을 이용해서 추출적 요약해보기
추상적 요약은 추출적 요약과는 달리 문장의 표현력을 다양하게 가져갈 수 있지만, 추출적 요약에 비해서 난이도가 높아요. 반대로 말하면 추출적 요약은 추상적 요약에 비해 난이도가 낮고 기존 문장에서 문장을 꺼내오는 것이므로 잘못된 요약이 나올 가능성이 낮아요.

Summa의 summarize를 사용하여 추출적 요약을 해보세요.

# 정리     
---        
1. 이미지 로드
2. DeepLab 모델 생성
3. pretrained weight 불러오기
4. 피사체 sementic segmentation
5. 배경 blur처리
6. 피사체, 배경 합성 

# 루브릭 평가
---
1. **Abstractive 모델 구성을 위한 텍스트 전처리 단계가 체계적으로 진행되었다.**   
    - 분석단계, 정제단계, 정규화와 불용어 제거, 데이터셋 분리, 인코딩 과정이 빠짐없이 체계적으로 진행되었다.


2. **텍스트 요약모델이 성공적으로 학습되었음을 확인하였다.**  
    - 모델학습이 안정적으로 수렴되었음을 그래프를 통해 확인하였으며, 실제 요약문과 유사한 요약문장을 얻을 수 있었다. 
    
     
3. **Extractive 요약을 시도해 보고 Abstractive 요약 결과과 함께 비교해 보았다.** 
    - 두 요약 결과를 문법완성도 측면과 핵심단어 포함 측면으로 나누어 비교분석 결과를 제시하였다.
   

    
# 회고
---
## 어려웠던 점 
- 모델에 대한 이해
    - 이미 학습된 가중치를 불러오기 때문에 그 구조에 대해서 자세하게 알기 힘들어 따로 여러 자료들을 찾아보았다.
- 사진을 촬영하는 방법
    - 참고 글을 계속해서 읽어보아도 제대로 이해되지는 않았다.
    
## 알게된 점 
- 스티커 사진 프로젝트([Exp.3](https://github.com/miinkang/Project_AIFFEL/blob/main/%5BE-03%5Dcamera_sticker_application.ipynb))를 할 때 사용했던, 여러 이미지(벡터)연산들을 다시 사용해보면서 개념을 확실하게 이해했다.
- openCV의 여러 함수들도 다시 학습할 수 있었다.
    
## 더 공부할 점
- 사진
    - 실제 카메라의 작동법에 더 알아두면 이미지 처리하는 데에 많은 인사이트를 얻을 수 있을 것이라고 생각한다.
    
## 시도한 것들
- openCV의 여러 함수들의 파라미터들을 변경해보면서 쓰임새를 익혔다.

## 다짐
- 모델이 왜 연구되었는지에 대한 배경을 아는 것이 확실히 모델의 구조를 이해하기 쉽게한다.